# TP4: Regression logistique (pédestre)

En fin de séance, votre TP devra être déposé sur le dépot:
https://synapse.math.univ-toulouse.fr/s/X4l9EeymOIEpfOm

**Attention: Je n'accepte pas les TPs d'élèves absents en cours.**

Nous nous intéressons au dataset public [Framingham](https://www.kaggle.com/amanajmera1/framingham-heart-study-dataset)
issu d'une étude en 1948 à Framingham, Massaschussets, USA et coordonnée par le U.S. Public Health Service. Je vous propose la version [ici](https://www.math.univ-toulouse.fr/~rchhaibi/teaching/2019/M1SID/framingham_fr.csv)
où j'ai simplement traduit les champs. Vous pouvez consultez les kernels disponibles, mais le but de ce TP est une implémentation plus manuelle et pédestre de la régression linéaire.

Description par le gouvernement américain: [Lien](https://biolincc.nhlbi.nih.gov/studies/framcohort/)
- Pour les variables binaires : “1”=“Oui”, “0”=“Non”.
- Pour les variables continues: Valeur intensive

Variable d'intérêt en dernière colonne:
Risque à 10 ans de développer une maladie coronaire (binaire)

* Facteurs démographiques:
  * Sexe: Masculin ou Feminin (binaire: "1"=Masculin)
  * Age: Continu
  * Education: Niveau d'éducation 1,2,3,4
* Facteurs comportementaux:
  * Fumeur: (binaire)
  * CigsParJour: Cigarette par jour
* Facteurs médicaux historiques / Historique médical:
  * meds (binaire): si le patient est traité pour des problèmes de pression sanguine
  * avc  (binaire): si le patient a déjà fait un avc
  * hypertension (binaire): si le patient a de l'hypertension
  * diabete (binaire): si le patient est diabétique
* Facteurs médicaux courants:
  * Tot Chol: niveau de cholesterol total HDL + LDL + VLDL (Continu)
  * Sys BP: pression sanguine systolique (Continu)
  * Dia BP: pression sanguine diastolique (Continu)
  * IMC: Indice de Masse Corporelle (Continu)
  * freqCardique: Fréquence cardiaque (Continu)
  * Glucose: niveau de glucose (Continu)

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## I. Echauffement: L'algorithme de Newton-Raphson

Exercice 1:
- Rappeler la récurrence de Newton-Raphson vue en cours, dans sa version scalaire
- Implémenter l'algorithme de Newton-Raphson, toujours dans sa version scalaire
- Commenter la vitesse de convergence

In [ ]:
# Version scalaire
def newton_raphson_scalar( x0, func, grad_func, n_max=100, eps=1e-8):
    x = x0 # TODO: Value
    k = 0  # TODO: Number of steps
    return (x, k)

In [ ]:
# Test pour la version scalaire
a    = 2
scalar_func = lambda x: x**2-a
scalar_grad = lambda x: 2*x

count  = 8
errors = []
print("Target value: ", np.sqrt(a))
print("NR value / Iteration count:")
for n_max in range(1,count):
    x, k = newton_raphson_scalar(1, scalar_func, scalar_grad, n_max=n_max)
    print(x, k)

## II. Exploration de données

Exercice 2:
- Lire le code suivant.
- Pourquoi elève-t-on la colonne "education"? Il s'agit pourtant (sans doute) d'un facteur pertinent.

In [ ]:
#Importation des données et nettoyage
dataframe=pd.read_csv("framingham_fr.csv")
dataframe.drop(['education'],axis=1,inplace=True)
dataframe.head()

#Counting the missing values and dropping them
count=0
for i in dataframe.isnull().sum(axis=1):
    if i>0:
        count=count+1
print("Le nombre total de lignes avec des valeurs manquantes est ", count)
print("Il s'agit de",round((count/len(dataframe.index))*100), '% du jeu de données.')
print("")
dataframe.dropna(axis=0,inplace=True)

#Ajout de la constante
from statsmodels.tools import add_constant as add_constant
dataframe = add_constant(dataframe)

# Statistiques descriptives
print("Statistiques descriptives:")
dataframe.describe()

In [ ]:
cols = dataframe.columns
# Définition de la variable d'intéret
Y = np.array( dataframe[cols[-1]] )
# Définition de la matrice des facteurs
X = np.array( dataframe[cols[:-1]] )
# Taille des matrices
print("dim X: ", X.shape)
print("dim Y: ", Y.shape)

## III. Implémentation de la régression logistique

Nous nous intéressons à l'estimation de 
$$ \mathbb{P}\left( Y=1 | X \right)$$
grâce à un modèle de régression logistique. On pourra se référer à la [page Wikipedia](https://fr.wikipedia.org/wiki/R%C3%A9gression_logistique#Le_mod%C3%A8le).

- La fonction de lien est définie pour $p = \mathbb{P}\left( Y = 1 | X \right) = \mathbb{E}\left( Y | X \right)$ par
$$ 
   \textrm{logit}( p )
 = \log\left( \frac{p}{1-p} \right)
 = \beta X \ ,
$$
où $\beta X$ est le produit scalaire entre les paramètres $\beta \in \mathbb{R}^k$ et les facteurs $X \in \mathbb{R}^k$. De façon équivalente:
$$ p = \frac{e^{\beta X}}{1+e^{\beta X}} \ .$$

- Dans ce cas, la log-vraisemblance s'écrit
$$
   \ell\left( y, x, \beta \right)
 = \frac{1}{n} \sum_{i=1}^n \left[
   y_i \left( \beta x_i \right)
   - \log (1+e^{\beta x_i})
   \right]
$$
Preuve:
$$
   \begin{align*}
   \ell\left( y, x, \beta \right) := & \frac{1}{n} \log \mathbb{P}\left( \forall i, \ Y_i = y_i | \forall i, \ X_i = x_i \right) \\
   = & \frac{1}{n} \sum_{i=1}^n \log \mathbb{P}\left( Y_i = y_i | X_i = x_i \right) \\
   = & \frac{1}{n} \sum_{i=1}^n 
   \left[
   y_i \log \mathbb{P}\left( Y_i = 1 | X_i = x_i \right)
   + (1-y_i) \log \mathbb{P}\left( Y_i = 0 | X_i = x_i \right)
   \right]
   \\
   = & \frac{1}{n} \sum_{i=1}^n \left[
   y_i \log \frac{e^{\beta x_i}}{1+e^{\beta x_i}}
   + (1-y_i) \log \frac{1}{1+e^{\beta x_i}}
   \right]
   \\
   = & \frac{1}{n} \sum_{i=1}^n \left[
   y_i \left( \beta x_i \right)
   + \log \frac{1}{1+e^{\beta x_i}}
   \right]
   \\
   = & \frac{1}{n} \sum_{i=1}^n \left[
   y_i \left( \beta x_i \right)
   - \log (1+e^{\beta x_i})
   \right] \ .
   \\
   & \qquad \qquad \qquad \qquad \qquad \qquad CQFD
   \end{align*}
$$
- En conséquence:
$$
   \nabla_\beta \ell\left( y, x, \beta \right)
 = \frac{1}{n} \sum_{i=1}^n \left(
   y_i
   -
   \frac{e^{\beta x_i}}{1+e^{\beta x_i}}
   \right) x_i
 = \frac{1}{n} \sum_{i=1}^n \left(
   y_i
   - 
   \frac{1}{1+e^{-\beta x_i}}
   \right) x_i
 = \frac{1}{n} V X \ ,
$$
$$
   \nabla_\beta^2 \ell\left( y, x, \beta \right)
 = \frac{1}{n} \sum_{i=1}^n 
   \frac{-e^{-\beta x_i}}{(1+e^{-\beta x_i})^2}
   x_i^T x_i 
 = \frac{1}{n} \sum_{i=1}^n 
   \frac{-1}{2 + e^{-\beta x_i} + e^{\beta x_i}}
   x_i^T x_i 
 = \ \frac{1}{n} X^T W X,
$$
où $V$ et $W$ sont des matrices explicites


Exercice 3:
Refaire les calculs ci-dessus et écrire trois fonctions
- ${\it logL}$: Calcul de la log vraisemblance (Fait)
- ${\it diff\_logL}$ : Calcul de son gradient
- ${\it diff2\_logL}$: Calcul de la matrice Hessienne


In [ ]:
def logL( beta, y, x):
    result  = 0
    betaX   = np.dot(x, beta)
    y_betaX = y*betaX
    result  = np.sum( y_betaX-np.log(1+np.exp(betaX)) )
    result  = result/len(y)
    return result


def diff_logL( beta, y, x):
    result  = 0
    # TODO
    return result

def diff2_logL( beta, y, x):
    result = 0
    # TODO
    return result


In [ ]:
# Test
k     = X.shape[1]
beta0 = np.random.rand(k)/100
print("Log-vraisemblance initiale: ", logL(beta0, Y, X))
print("Gradient  initial : ", diff_logL(beta0, Y, X))
print("Hessienne initiale: ", diff2_logL(beta0, Y, X).shape )


Exercice 4:
- Ecrire la version vectorielle de Newton-Raphson donnée en cours
- Fitter le modèle en maximisant la log vraisemblance par l'algorithme de Newton-Raphson

In [ ]:
# Version vectorielle de Newton-Raphson
def newton_raphson( x0, func, jacobian_func, n_max=100, eps=1e-8, optional=None):
    x = np.zeros(len(x0))
    k = 0
    # TODO
    return (x, k)

# Fitting
k     = X.shape[1]
beta0 = np.zeros(k)
l     = lambda beta : logL(beta, Y, X)
diff  = lambda beta : diff_logL(beta, Y, X) 
diff2 = lambda beta : diff2_logL(beta, Y, X) 
result = newton_raphson( beta0, diff, diff2, optional=l)
print("")

#Final values
print(" Likelihood: %f \n Number of iterations: %d \n Estimated beta:" % (l(result[0]), result[1]), result[0] )

## IV. Comparaison à statmodels

Bien entendu, ce que vous venez d'implémenter est standard et est contenu dans les packages python appropriés. 

Comparez vos résultats à:

In [ ]:
import statsmodels.api as sm

cols=dataframe.columns[:-1]
model=sm.Logit(dataframe.risque10ans,dataframe[cols])
result=model.fit()
result.summary()